In [18]:
from konlpy.tag import Okt

okt = Okt()

stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', '방이', '제대로',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', '그렇다',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다',
    '좀', '너무', '정말', '많이', '조금',
    '사장', '이용', '용하다', '물이',
    '뿐', '대로', '만', '따름', '나름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안','그냥',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

stopwords = set(w.lower() for w in stopwords)


def tokenize(text):
    try:
        return [
            word for word, pos in okt.pos(text, stem=True)
            if pos in ['Noun', 'Adjective']
            and word not in stopwords
            and len(word) > 1
        ]
    except:
        return []

In [ ]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 데이터 로딩 및 정제
train_df = pd.read_csv("36000_reviews_label.csv", encoding="utf-8-sig")
train_df = train_df[train_df['label'].isin([0, 1])]
X_train = train_df["sentence"]
y_train = train_df["label"]

# 모델 A: CountVectorizer + LogisticRegression
vectorizer_a = CountVectorizer()
X_train_a = vectorizer_a.fit_transform(X_train)
model_a = LogisticRegression(max_iter=1000, class_weight='balanced')
model_a.fit(X_train_a, y_train)

# 모델 B: TF-IDF(ngram) + LogisticRegression
vectorizer_b = TfidfVectorizer(ngram_range=(1, 2))
X_train_b = vectorizer_b.fit_transform(X_train)
model_b = LogisticRegression(max_iter=1000, class_weight='balanced')
model_b.fit(X_train_b, y_train)

# 저장
joblib.dump(model_a, 'model_logistic_count.pkl')
joblib.dump(vectorizer_a, 'vectorizer_count.pkl')

joblib.dump(model_b, 'model_logistic_tfidf.pkl')
joblib.dump(vectorizer_b, 'vectorizer_tfidf.pkl')

print("✅ 두 모델 저장 완료")


✅ 두 모델 저장 완료


In [3]:
import pandas as pd

df = pd.read_csv("36000_reviews.csv", encoding="utf-8-sig")
texts = df['sentence'].fillna('').astype(str)

model_a = joblib.load("models/model_logistic_count.pkl")
vectorizer_a = joblib.load("models/vectorizer_count.pkl")

X_new_a = vectorizer_a.transform(texts)
df['pred_count'] = model_a.predict(X_new_a)
df['prob_count'] = model_a.predict_proba(X_new_a)[:, 1]

In [4]:
model_b = joblib.load("models/model_logistic_tfidf.pkl")
vectorizer_b = joblib.load("models/vectorizer_tfidf.pkl")

X_new_b = vectorizer_b.transform(texts)
df['pred_tfidf'] = model_b.predict(X_new_b)
df['prob_tfidf'] = model_b.predict_proba(X_new_b)[:, 1]

In [5]:
df['sentiment_count'] = df['pred_count'].map({0: '부정', 1: '긍정'})
df['sentiment_tfidf'] = df['pred_tfidf'].map({0: '부정', 1: '긍정'})

df.to_csv("숙소_리뷰_감성_분석_결과.csv", encoding="utf-8-sig", index=False)
print(df[['sentence', 'sentiment_count', 'sentiment_tfidf']].head())

                                            sentence sentiment_count  \
0                           따뜻하고 좋아요 특히 사장님이 너무 친절해요              긍정   
1  시설 좋은 곳 사용하려면 무조건 여기로 오는 것 같아요 무한 대실도 다음에 이용 예...              부정   
2                            아주 만족합니다 시설 인테리어 아주 좋아요              부정   
3                             진짜로 좋아요 아주 자주 이용하고 어 오              긍정   
4                   숙소가 깔끔하고 사장님도 친절하고 좋았어요 잘 쉬다 갑니다              긍정   

  sentiment_tfidf  
0              긍정  
1              긍정  
2              긍정  
3              긍정  
4              긍정  


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler

# 파일 로드
file_path = "숙소_리뷰_감성_분석_결과.csv"
df = pd.read_csv(file_path)

# 기본 컬럼 이름 정리
df.columns = df.columns.str.strip()
df.rename(columns={
    'name': 'accommodation',
    'sentence': 'review',
    'sentiment_tfidf': 'sentiment'  # 이 라인!

}, inplace=True)

# 필수 컬럼만 추출
df = df[['accommodation', 'review', 'sentiment']]

# 1. 숙소별 감성 통계 요약
summary = df.groupby(['accommodation', 'sentiment'])['review'].count().unstack(fill_value=0)
summary['total'] = summary.sum(axis=1)

# 감성 비율 계산
for label in ['긍정', '부정', '중립']:
    if label in summary.columns:
        summary[f'{label}_ratio'] = (summary[label] / summary['total'] * 100).round(1)
    else:
        summary[label] = 0
        summary[f'{label}_ratio'] = 0.0

# 2. 감성별 예시 문장 추출 함수
def extract_examples(df, label, n=2):
    return df[df['sentiment'] == label].groupby('accommodation')['review'].apply(lambda x: x.head(n).tolist())

# 예시 문장 추가
summary['긍정_예시'] = extract_examples(df, '긍정')
summary['부정_예시'] = extract_examples(df, '부정')
summary['중립_예시'] = extract_examples(df, '중립')

# 결측치 보완
summary.fillna({'긍정_예시': '', '부정_예시': '', '중립_예시': ''}, inplace=True)

# 필요하면 저장
summary.to_csv("숙소별_감성분석_요약.csv", encoding='utf-8-sig')

# 일부 미리보기
print(summary[['긍정', '부정', '중립', '긍정_ratio', '부정_ratio', '중립_ratio', '긍정_예시', '부정_예시']].head())

sentiment           긍정   부정  중립  긍정_ratio  부정_ratio  중립_ratio  \
accommodation                                                   
라한호텔 전주            249   35   0      87.7      12.3       0.0   
베스트웨스턴 플러스 전주 호텔  1086  312   0      77.7      22.3       0.0   
솔 호텔               253   53   0      82.7      17.3       0.0   
완주 스페이스셔틀 전주        12    0   0     100.0       0.0       0.0   
전주 관광호텔           1158  592   0      66.2      33.8       0.0   

sentiment                                                     긍정_예시  \
accommodation                                                         
라한호텔 전주           [총평 아름다운 한옥마을뷰를 자랑하는 전주 최고의 라한호텔 전주 위치 라한호텔 위치...   
베스트웨스턴 플러스 전주 호텔  [아들이랑 둘이서 축구보고와서 씻고 잠만잘거라 가성비좋은 베스트웨스턴플러스 호텔입니...   
솔 호텔              [침대 뒤로 창 나있는 게 따스해서 기분 좋았습니다 소파도 푹신하고 침대랑 소파 단...   
완주 스페이스셔틀 전주      [기대 이상입니다 일단 결과부터 말씀드리자면 사장님 굉장히 친절하시고 객실 청결도 ...   
전주 관광호텔           [숙소가 깔끔하니 좋았어요 컨디션도 좋았구요 다만 모기가 두마리 있어서 밤새 뜯 그...   

sentiment                                     

In [15]:
# 확률 기반 + 키워드 기반 중립 보정 로직을 적용
df = pd.read_csv('숙소_리뷰_감성_분석_결과.csv', encoding='utf-8-sig')
# 중립 판단 기준 키워드 리스트
neutral_clues = ['무난', '그냥', '나쁘지', '괜찮긴', '보통', '가격 대비', '애매', '글쎄', '평범', '그럭저럭']

# 숫자 예측값을 텍스트 라벨로 변환
def convert_label(pred):
    if pred == 1 or pred == '1':
        return '긍정'
    elif pred == 0 or pred == '0':
        return '부정'
    return str(pred)

df['pred_tfidf'] = df['pred_tfidf'].apply(convert_label)

# 중립 보정 함수
def smart_reclassify(row, threshold_diff=0.1, prob_threshold=0.6):
    prob = row['prob_tfidf']
    pred = row['pred_tfidf']
    oppo_prob = 1 - prob
    diff = abs(prob - oppo_prob)

    prob_based = (diff < threshold_diff) or (max(prob, oppo_prob) < prob_threshold)
    review = str(row['sentence'])
    keyword_based = any(kw in review for kw in neutral_clues)

    if prob_based or keyword_based:
        return '중립'
    return pred

# 새 컬럼 추가
df['smart_reclassified'] = df.apply(smart_reclassify, axis=1)

# 저장
df.to_csv('중립_보정_결과.csv', encoding='utf-8-sig', index=False)


In [14]:
df.to_csv('중립_보정_결과.csv', encoding='utf-8-sig', index=False)

In [17]:
df = pd.read_csv('중립_보정_결과.csv', encoding='utf-8-sig')
df[['sentence', 'smart_reclassified']].to_csv('중립_보정_결과_내용만.csv', encoding='utf-8-sig', index=False)